In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-02 15:46:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.96MB/s    in 0.2s    

2021-12-02 15:46:52 (5.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load Amazon Data into Spark df
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RY01SAV7HZ8QO|          4|            0|          0|   N|                Y|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|                N|
|R149Q3B5L33NN5|          5|            0|          0|   N|                Y|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|                Y|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
vine_filtered = vine_df.filter('total_votes>=20')
vine_filtered.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
|R1LV778IDGRJQ6|          1|            4|         23|   N|                Y|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R2LBEAGVJA7HK1|          3|            5|         48|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R3QLX5NODD1OJ4|          1|            9|         25|   N|                Y|
|R2SOOHE0XUQON1|          1|            2|         24|   N|     

In [7]:
helpful_votes = vine_filtered.filter('helpful_votes/total_votes>=0.5')
helpful_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

In [8]:
paid_vine = helpful_votes.filter(helpful_votes['vine']=='Y')
paid_vine.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LLICBKRHLRJC|          4|           17|         22|   Y|                N|
| RUD135S06DEVE|          5|           41|         51|   Y|                N|
|R2WG8UL4SGDPDH|          4|           23|         25|   Y|                N|
|R158AOKOA28WS2|          4|           18|         28|   Y|                N|
| RGWV3QLJQXU9X|          5|          332|        380|   Y|                N|
| RD0E1RZSSYGGN|          5|          261|        292|   Y|                N|
|R3LGCTQH5ZMGO0|          5|           61|         68|   Y|                N|
|R2H24YTRLARCZJ|          4|          277|        297|   Y|                N|
| RYYTC9JTG7WPW|          5|          107|        134|   Y|                N|
| RTB8H4YQYTLY3|          4|           58|         62|   Y|     

In [9]:
unpaid_vine = helpful_votes.filter(helpful_votes['vine']=='N')
unpaid_vine.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

In [31]:
# Total number of paid and unpaid reviews

paid_vine_count = paid_vine.count()
print('Total count of "helpful" paid Vine reviews is: ', paid_vine_count)

unpaid_vine_count = unpaid_vine.count()
print('Total count of "helpful" unpaid Vine reviews is: ', unpaid_vine_count)

Total count of "helpful" paid Vine reviews is:  261
Total count of "helpful" unpaid Vine reviews is:  24040


In [33]:
# The number of 5-star reviews of paid and unpaid reviews

paid_5star = paid_vine.filter(paid_vine['star_rating']==5).count()
print('The number of "helpful" paid Vine reviews is: ', paid_5star)

unpaid_5star = unpaid_vine.filter(unpaid_vine['star_rating']==5).count()
print('The number of "helpful" unpaid Vine reviews is: ', unpaid_5star)

The number of "helpful" paid Vine reviews is:  106
The number of "helpful" unpaid Vine reviews is:  10899


In [34]:
# The percentage of 5-star reviews for both paid and unpaid 

paid_percentage = paid_5star / paid_vine_count * 100
paid_percentage = round(paid_percentage, 2)
print('The percentage of 5-star reviews of "helpful" paid Vine program is: ', paid_percentage, '%')

unpaid_percentage = unpaid_5star / unpaid_vine_count * 100
unpaid_percentage = round(unpaid_percentage, 2)
print('The percentage of 5-star reviews of "helpful" unpaid Vine program is: ', unpaid_percentage, '%')

The percentage of 5-star reviews of "helpful" paid Vine program is:  40.61 %
The percentage of 5-star reviews of "helpful" unpaid Vine program is:  45.34 %


In [17]:
# total_reviews = helpful_votes.count()
# print('The total number of helpful reviews is: ', total_reviews)

In [18]:
# total_5star = helpful_votes.filter(helpful_votes['star_rating'] == 5).count()
# print('The number of 5-star reviews is: ', total_5star)